# Syntax tutorial

This notebook is a terse tutorial walkthrough of the syntax of the `guidance` language (which is based on the Handlebars templating language). It is not complete, but other than the <a href="https://guidance.readthedocs.org">full documentation</a>, it is a good reference.

In [1]:
import sys
sys.path.append("./src")
from model_dict import *
from inference import *
import pandas as pd
import guidance

/opt/conda/envs/Eval4NLP23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# WizardLM-13B-V1.1-GPTQ
# Nous-Hermes-13b
# guanaco-65B-GPTQ

model_name = 'Nous-Hermes-13b'
model_path = f'/Workspace/jh/SharedTask2023/models/{model_name}'

model, tokenizer, u_prompt, a_prompt = load_from_catalogue(model_name, model_path, device="cuda:1")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
g_model = guidance.llms.Transformers(
            model, tokenizer=tokenizer, trust_remote_code=True,
)
guidance.llms.Transformers.cache.clear()

In [4]:
# we will use GPT-3 for most of the examples in this tutorial
guidance.llm = g_model

# ToT test

In [5]:
sport_news =  """
Bayern Munich have signed South Korean centre-back Kim Min-jae from Napoli on a five-year deal.
Kim, 26, was an integral member of the Napoli squad that ended a 33-year wait for a Serie A title last season.
He was named the best defender in Serie A after making 35 league appearances in his debut campaign in Italy.
"Bayern is a dream for every footballer," said Kim, who has 49 international caps. " I'm really looking forward to what's to come"
"It's a new beginning for me. I'll continue to develop here. In discussions with the club, it was made clear to me from the start how interested they are in me."
Thomas Tuchel's Bayern won the Bundesliga on goal difference last season and the former Chelsea boss has been keen to make improvements to his squad with the club linked with Tottenham striker Harry Kane.
Bayern reportedly paid a release clause of 50m euros (£43m) to sign Kim.
Kim started his career in the third tier of South Korean football, reaching the top-flight and played in the Chinese Super League before making the switch to Europe.
He joined Napoli from Turkish side Fenerbahce in the summer of 2022.
"""

In [6]:
business_news = '''
British microchip designing giant Arm has announced it has filed paperwork to sell its shares in the US.

The Cambridge-based company, which designs chips for devices from smartphones to game consoles, plans to list on New York's Nasdaq in September.

Arm did not reveal the number of shares for sale or the price, but its proposed initial public offering (IPO) could be the biggest listing since late 2021.

In March, in a blow to the UK, the firm opted against listing shares in London.

On Monday, Arm announced that it had now publicly filed a registration statement relating to a proposed IPO. It said the number of shares to be offered and the price range for them were yet to be determined.

But the company is reportedly looking for a valuation of between $60bn (£47bn) to $70bn.

Arm was bought in 2016 by Japanese conglomerate Softbank in a deal worth £23.4bn. Prior to the takeover, it was listed in both London and New York for 18 years.

Its chip design instructions and technologies are used by manufacturers like the Taiwan Semiconductor Manufacturing Company and technology giants Apple and Samsung to make their own chips.

Listing a firm on a stock exchange takes it from being a private firm to a public company, with investors able to buy and sell shares of a company's stock on specific exchanges.

Reports previously suggested the firm had sought to raise between $8bn and $10bn through the listing on the technology-heavy Nasdaq platform. Other major technology companies including Google, Apple and Facebook trade on the Nasdaq.
'''

In [7]:
news_categories = [
    "World",
    "Business",
    "Politics",
    "Health",
    "Education",
    "Science",
    "Technology",
    "Entertainment",
    "Arts",
    "Sports"
]

In [25]:
guidance.llm.cache.clear()
torch.cuda.empty_cache()

In [ ]:
'''
[{{#geneach 'items' num_iterations=3}}
        "{{gen 'this'}}",{{/geneach}}]
        '''

In [27]:
guidance.llm.cache.clear()

In [41]:
guidance.llm.cache.clear()

oneshot_program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

Generate three strategies to summarize {{category}} news which has content like "{{summary}}" 
Strategy 1: {{gen "strategy" list_append=True}}
Strategy 2: {{gen "strategy" list_append=True}}
Strategy 3: {{gen "strategy" list_append=True}}

{{strategy}}

News:{{news}}
# {{#each strategy}}
Summarize the news based on the strategy below.
Strategy: {{this}}
Summarization: {{gen 'summarization' list_append=True}}

{{/each}}
{{'summarization'}}
''')

executed_program = oneshot_program(news=sport_news, 
                                   options=news_categories,
                                   prompt_placeholder=u_prompt,
                                   response_placeholder=a_prompt)()
# executed_program()
# guidance.llm.cache.clear()


Stop program 
News: 
British microchip designing giant Arm has announced it has filed paperwork to sell its shares in the US.

The Cambridge-based company, which designs chips for devices from smartphones to game consoles, plans to list on New York's Nasdaq in September.

Arm did not reveal the number of shares for sale or the price, but its proposed initial public offering (IPO) could be the biggest listing since late 2021.

In March, in a blow to the UK, the firm opted against listing shares in London.

On Monday, Arm announced that it had now publicly filed a registration statement relating to a proposed IPO. It said the number of shares to be offered and the price range for them were yet to be determined.

But the company is reportedly looking for a valuation of between $60bn (£47bn) to $70bn.

Arm was bought in 2016 by Japanese conglomerate Softbank in a deal worth £23.4bn. Prior to the takeover, it was listed in both London and New York for 18 years.

Its chip design instructions and technologies are used by manufacturers like the Taiwan Semiconductor Manufacturing Company and technology giants Apple and Samsung to make their own chips.

Listing a firm on a stock exchange takes it from being a private firm to a public company, with investors able to buy and sell shares of a company's stock on specific exchanges.

Reports previously suggested the firm had sought to raise between $8bn and $10bn through the listing on the technology-heavy Nasdaq platform. Other major technology companies including Google, Apple and Facebook trade on the Nasdaq.
 
Select a category for a given news and create a one-sentence summary of the news.
Category: Business 
One-sentence-summary: Arm, a British microchip designing giant, has announced it has filed paperwork to sell its shares in the US. 

Generate a strategy to summarize Business news which has content like " Arm, a British microchip designing giant, has announced it has filed paperwork to sell its shares in the US. " 
Strategy: <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_c24d369a4ca34232ad14279a42c3764e(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("c24d369a4ca34232ad14279a42c3764e_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("c24d369a4ca34232ad14279a42c3764e_" + (i+1)); if (!next_el) { next_el = document.getElementById("c24d369a4ca34232ad14279a42c3764e_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 3) + 1) + "/" + 3;}cycle_c24d369a4ca34232ad14279a42c3764e(this);'>1/3 Describe the company, its significance in the industry, the key developments or reasons that make the news important or interesting. 
1. Identify the key player: Arm, the British microchip designing giant.
2. Identify the action: Filed paperwork to sell shares.
3. Identify the location: US.
4. Create a concise summary: Arm, a UK microchip designer, has filed paperwork to sell its shares in the US. 
1. Find the key information in the news such as the company name, its purpose, and the action it has taken. 

2. Determine the appropriate category for this news which is Business. 

3. Create a catchy and concise summary that is informative, yet easy to read, by conveying the most essential details while keeping it short and precise. 

4. Use appropriate vocabulary and sentence structure to convey the information in a clear and accurate manner. 

5. Double-check the summary to ensure that it accurately reflects the news, and make necessary adjustments if needed. 
 ['Describe the company, its significance in the industry, the key developments or reasons that make the news important or interesting.', '\n1. Identify t

In [42]:
executed_program.variables()

{'llm': <guidance.llms._transformers.Transformers at 0x7fa3282eaef0>,
 'logging': False,
 'news': "\nBritish microchip designing giant Arm has announced it has filed paperwork to sell its shares in the US.\n\nThe Cambridge-based company, which designs chips for devices from smartphones to game consoles, plans to list on New York's Nasdaq in September.\n\nArm did not reveal the number of shares for sale or the price, but its proposed initial public offering (IPO) could be the biggest listing since late 2021.\n\nIn March, in a blow to the UK, the firm opted against listing shares in London.\n\nOn Monday, Arm announced that it had now publicly filed a registration statement relating to a proposed IPO. It said the number of shares to be offered and the price range for them were yet to be determined.\n\nBut the company is reportedly looking for a valuation of between $60bn (£47bn) to $70bn.\n\nArm was bought in 2016 by Japanese conglomerate Softbank in a deal worth £23.4bn. Prior to the tak

In [ ]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')

In [ ]:
strategy_objective = 'Generate a strategy to summarize {{category}} news which has content like "{{summary}}"'
strategy_program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

{{strategy_objective}}
Strategy: {{gen "strategies" n=5 temperature=0.7}}
''')


executed_program = strategy_program(news=news, strategy_objective=strategy_objective, options=options)

In [ ]:
strategy_objective = 'Generate a strategy to summarize {{category}} news which has content like "{{summary}}"'
strategy_program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

{{strategy_objective}}
Strategy: {{gen "strategies" n=5 temperature=0.7}}
''')


executed_program = strategy_program(news=news, strategy_objective=strategy_objective, options=options)


In [ ]:
evaluation_objective = 'Score pessimistically the input based on the objective on a continuous scale from 0 to 10, where a score of zero means\
irrelevant, factually incorrect and not readable" and score of ten means\
relevant, factually correct, good readability.'
# evaluation_objective = 'Score pessimistically the input based on the objective on a continuous scale from 0 to 100, where a score of zero means\
# irrelevant, factually incorrect and not readable" and score of one hundred means\
# relevant, factually correct, good readability.'
evaluation_program = guidance('''
Objective: {{evaluation_objective}}
Input: {{input}}
Score: {{gen 'score' pattern='(10|[0-9])'}}
''')

In [ ]:
total_score = []
for strategy in executed_program["strategies"]:
    executed_eval_program = evaluation_program(input=strategy, evaluation_objective=evaluation_objective)
    total_score.append(executed_eval_program['score'])
    guidance.llm.cache.clear()

In [ ]:
total_score

In [ ]:
program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

Generate a strategy to summarize {{category}} news which has content like "{{summary}}" 
Strategy: {{gen "strategy" temperature=0.7 n=3}}

News:{{news}}
{{#each strategy}}
Summarize the news based on the strategy below.
Strategy: {{this}}
Summarization: {{gen 'summarization'}}

{{/each}}

''')

executed_program = program(news=news, options=options)

In [ ]:
executed_program['summarization']

In [ ]:
program = guidance('''
News:{{news}}
Select a category for a given news and create a one-sentence summary of the news.
Category: {{select "category" options=options}}
One-sentence-summary:{{gen "summary"}}

Generate a strategy to summarize {{category}} news which has content like "{{summary}}" 
Strategy: {{gen "strategy" temperature=0.7 n=3}}

{{#each strategy}}Strategy: {{this}}
Score the strategy based on how appropriate it is to summarize {{category}} news which has content like "{{summary}}"
Score: {{gen 'score' pattern='(100|[1-9]?[0-9])'}}

{{/each}}

''')

executed_program = program(news=news, options=options, silent=True)

In [ ]:
# it is often useful to execute a part of the program, but then not include that part in later context
# given to the language model. This can be done using the hidden=True argument. Several commands support
# hidden=True, but here we use the {{#block}} command (which is just a generic block command that does
# nothing other than what the arguments you pass to it do)
program = guidance('''Based on the category and facts of a given news story, generate summary approache to summarize the news in a brief but compelling way:
{{#block hidden=True}}{{news}}{{/block}}
Strategies:{{gen "strategies" temperature=1 n=3}}
I will show you a news and three straties for summarizing the news, and you should score each of them on a continuous scale from 0 to 100
focusing on how appropriate it is for making brief and informative summarization and call them score_1, score_2 and score_3.
News: {{news}}.
Strategies: {{strategies}}

''')

executed_program = program(news=news)

In [ ]:
executed_program['logprobs']

Silent execution

In [ ]:
# if you want to run a program without displaying the output, you can use the silent=True argument
executed_program = program(email='I hate tacos', silent=True)
executed_program['answer']

Generating with `n>1`

In [ ]:
# the {{gen}} command the n=number argument to generate multiple completions
# only the first completion is used for future context, but the variable set
# by the command is a list of all the completions, and you can interactively
# click through each completion in the notebook visualization
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7}}''')
executed_program = program()

In [ ]:
executed_program["best"]

Calling custom user defined functions

In [ ]:
# all the built in commands are functions from guidance.library.* but you can also pass in your own functions
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])

# note that we use hidden=True to prevent the {{gen}} command from being included in the output, and instead
# just use the variable it sets as an input to the aggregate function
program = guidance('''The best thing about the beach is{{gen 'best' n=3 temperature=0.7 max_tokens=7 hidden=True}}
{{aggregate best}}''')
executed_program = program(aggregate=aggregate)

Await

In [ ]:
# sometimes you want to partially execute a program, the `await` command allows you to do this
# it awaits a variable and then consumes that variables (so after the await command the variable)
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)

# note how the executed program is only partially executed, it stops at the await command
# because the instruction variable is not yet set
prompt = prompt(email='Hello there')


In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Portuguese.')
prompt2

In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Chinese.')
prompt2

## Chat

In [ ]:
# to use role based chat tags you need a chat model, here we use gpt-3.5-turbo but you can use 'gpt-4' as well
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

In [ ]:
# note that we enclose all of the text in one of the valid role tags for the model
# `system`, `user`, and `assistant` are just shorthand for {{#role name="system"}}...{{/role}}
# the whitepace outside the role tags is ignored by gpt-4, the whitespace inside the role tags is not
# so we use the ~ to remove the whitespace we don't want to give to the model (but want to keep in the code for clarity)
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{! this is a comment. note that we don't have to use a stop="stop_string" for the gen command below because Guidance infers the stop string from the role tag }}
{{#assistant~}}
{{gen 'response'}}
{{~/assistant}}''')

executed_program = program(conversation_question='What is the meaning of life?')

Multistep

In [ ]:
# you can create and guide multi-turn conversations by using a series of role tags
experts = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}

{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
                   
experts(query='What is the meaning of life?')

With hidden

In [ ]:
# if you want the model to have some inner dialog but then not include that dialog
# in the context of later generations, you can use the {{#block}} command with hidden=True
program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#block hidden=True}}
{{#user~}}
Please tell me a joke
{{~/user}}

{{! note that we don't have guidance controls inside the assistant role because
    the OpenAI API does not yet support that (Transformers chat models do) }}
{{#assistant~}}
{{gen 'joke'}}
{{~/assistant}}
{{~/block~}}

{{#user~}}
Is the following joke funny? Why or why not?
{{joke}}
{{~/user}}

{{#assistant~}}
{{gen 'funny'}}
{{~/assistant}}''')
program()


Agents

In [ ]:
# by putting an `await` inside a `geneach` loop you can create agents that consume some
# varable, then do something and then wait for more content
program = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{~#geneach 'conversation' stop=False}}
{{#user~}}
{{set 'this.user_text' (await 'user_text')}}
{{~/user}}

{{#assistant~}}
{{gen 'this.ai_text' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')
program = program(user_text ='hi there')

In [ ]:
# as we go through the loop we build up a conversation variable that contains the history of the conversation
# note that the last entry in the conversation variable is empty because the `await` call happens before any
# content is added to the `this` variable that represents the current item in the geneach loop
program['conversation']

In [ ]:
# here we call the agent again and the loop continues, in this case building out a conversation
program = program(user_text = 'What is the meaning of life?')

In [ ]:
program['conversation']

Using tools

The example below uses a search engine (or a mock of one) to answer user questions. The whole system is defined in a single `guidance` program, but you could also break it into multiple programs and `await` external calls if you don't want the guidance program to control the whole process.

In [ ]:
def is_search(completion):
    return '<search>' in completion

def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
        'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
        {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
        'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
        {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
        'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]

search_demo = guidance('''Seach results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}''')

demo_results = [
    {'title': 'OpenAI - Wikipedia', 'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
    {'title': 'About - OpenAI', 'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'}, 
    {'title': 'Ilya Sutskever | Stanford HAI', 'snippet': '''Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.'''}
]

s = search_demo(results=demo_results)

practice_round = [
    {'role': 'user', 'content' : 'Who are the founders of OpenAI?'},
    {'role': 'assistant', 'content': '<search>Who are the founders of OpenAI</search>'},
    {'role': 'user', 'content': str(search_demo(results=demo_results))},
    {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.'},
]

program = guidance('''
{{#system~}}
You are a helpful assistant.
{{~/system}}

{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}

{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}

{{#each practice}}
{{#if (== this.role "user")}}
{{#user}}{{this.content}}{{/user}}
{{else}}
{{#assistant}}{{this.content}}{{/assistant}}
{{/if}}
{{/each}}

{{#user~}}
That was great, now let's do another one.
{{~/user}}

{{#assistant~}}
Sounds good
{{~/assistant}}

{{#user~}}
{{user_query}}
{{~/user}}

{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}

{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
''')

query = "What is Facebook's stock price right now?"

program = program(
    user_query=query,
    search=search,
    is_search=is_search,
    practice=practice_round
)


<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>